Import all libraries

In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD
import torch.nn.functional as F

Define the neural network

In [ ]:
class LeNet(nn.Module):
    def __init__(self, num_classes=30):
        super(LeNet, self).__init__()
        # 32x32
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 16x16
        self.conv2 = nn.Conv2d(in_channels=16*16*16, out_channels=32, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        #8x8
        self.dense1 = nn.Linear(in_features=8*8*32, out_features=256)
        self.relu3 = nn.ReLU()
        
        self.dense2 = nn.Linear(in_features=256, out_features=num_classes)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = nn.Dropout2d(p=0.25)
        
        x = x.view(-1, 16*16*16)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = nn.Dropout2d(p=0.25)
        
        x = x.view(-1, 8*8*32)
        x = self.dense1(x)
        x = self.relu3(x)
        x = nn.Dropout2d(p=0.25)
        
        x = self.dense2(x)
        x = self.softmax(x)
        return x
    
    def name(self):
return "LeNet"

Process images

In [ ]:
'''prcess images'''

 Instantiate the model, create the optimizer and loss function

In [ ]:
cuda_avail = torch.cuda.is_available()

model = LeNet(num_classes=30)
if cuda_avail:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01) #, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

Function to adjust learning rates

In [ ]:
def adjust_learning_rate(epoch):
    lr = 0.001

    if epoch > 180:
        lr = lr / 1000000
    elif epoch > 150:
        lr = lr / 100000
    elif epoch > 120:
        lr = lr / 10000
    elif epoch > 90:
        lr = lr / 1000
    elif epoch > 60:
        lr = lr / 100
    elif epoch > 30:
        lr = lr / 10

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

Function to save model

In [ ]:
def save_models(epoch):
    torch.save(model.state_dict(), "LeNetmodel_{}.model".format(epoch))
        print("Checkpoint saved")

Function to evaluate model

In [ ]:
def test():
    model.eval()
    test_acc = 0.0
    for i, (images, labels) in enumerate(test_loader):

        if cuda_avail:
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

        # Predict classes using images from the test set
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        
        test_acc += torch.sum(prediction == labels.data)

    # Compute the average acc and loss over all 10000 test images
    test_acc = test_acc / 10000

    return test_acc

Training function

In [ ]:
def train(num_epochs):
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_acc = 0.0
        train_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # Move images and labels to gpu if available
            if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

            # Clear all accumulated gradients
            optimizer.zero_grad()
            # Predict classes using images from the test set
            outputs = model(images)
            # Compute the loss based on the predictions and actual labels
            loss = loss_fn(outputs, labels)
            # Backpropagate the loss
            loss.backward()

            # Adjust parameters according to the computed gradients
            optimizer.step()

            train_loss += loss.cpu().data[0] * images.size(0)
            _, prediction = torch.max(outputs.data, 1)
            
            train_acc += torch.sum(prediction == labels.data)

        # Call the learning rate adjustment function
        adjust_learning_rate(epoch)

        # Compute the average acc and loss over all 50000 training images
        train_acc = train_acc / 50000
        train_loss = train_loss / 50000

        # Evaluate on the test set
        test_acc = test()

        # Save the model if the test acc is greater than our current best
        if test_acc > best_acc:
            save_models(epoch)
            best_acc = test_acc

        # Print the metrics
        print("Epoch {}, Train Accuracy: {} , TrainLoss: {} , Test Accuracy: {}".format(epoch, train_acc, train_loss, test_acc)
